In this guided project, we will use the `stack` data structure to implement an algorithm that can evaluate numerical expressions. First off, let's define our classes -- nodes, linked lists and stacks -- and write their methods.

In [1]:
class Node:
    
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

In [2]:
class LinkedList:

    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.length += 1

    def __iter__(self):
        self._iter_node = self.head
        return self 

    def __next__(self):
        if self._iter_node is None:
            raise StopIteration
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        return ret

    def prepend(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        self.length += 1

    def __len__(self):
        return self.length

    def __str__(self):
        return str([value for value in self])

In [3]:
class Stack(LinkedList):
    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data
    
    def pop(self):
        tail = self.tail.data
        if self.length == 1: self.head = self.tail = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return tail
    
    def pop2(stack):
        return stack.pop(), stack.pop()

The inputs will be mathematical expressions in string form. The first step is to tokenize them into a list that will be iterated over. There's probably a more efficient way of doing this, but the function below does the job (note that we can't just use the `str.split()` method because sometimes there won't be a space between a number and an operator).

In [4]:
import re

def tokenize(expression):
    non_split_chars = [str(x) for x in list(range(10))] + ['.']
    splits = [c in non_split_chars for c in expression]
    new_expression = ''
    mini_expression = ''
    for idx, b in enumerate(splits):
        if b: mini_expression += expression[idx]
        else:
            new_expression += mini_expression + ' ' + expression[idx] + ' '
            mini_expression = ''
    if b: new_expression += expression[-1]
    new_expression = re.sub(' +', ' ', new_expression)
    new_expression = re.sub('\\* \\*', '**', new_expression)
    return new_expression.split()

When we write an expression, we use infix notation, meaning that we put the operators between the two operands. For a computer, it's much easier to evaluate an expression written in postfix notation. In postfix notation, the operands appear before the operator. The infix expression `1 + 2` becomes `1 2 +` in postfix notation.

We can evaluate an expression in postfix notation using a stack. We read the expression from left to right and do the following:

1. If we find a number, we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result.

First, we define the functions that will perform individual mathematical operations based on the above. Then, we create two dictionaries that associate the operators with the functions, and define the precedence level (remember BEDMAS?).

In [5]:
def process_minus(stack):
    top, second_to_top = stack.pop2()
    result = second_to_top - top
    stack.push(result)
    
def process_plus(stack):
    top, second_to_top = stack.pop2()
    result = second_to_top + top
    stack.push(result)

def process_times(stack):
    top, second_to_top = stack.pop2()
    result = second_to_top * top
    stack.push(result)

def process_divide(stack):
    top, second_to_top = stack.pop2()
    result = second_to_top / top
    stack.push(result)

def process_pow(stack):
    top, second_to_top = stack.pop2()
    result = second_to_top ** top
    stack.push(result)

precedence = {'+':1, '-':1, '*':2, '/': 2, '**':3}
operators = {'+': process_plus, '-': process_minus, '/': process_divide, '*': process_times, '**': process_pow}

Now, we write a function that can evaluate an expression in postfix form.

In [6]:
import pandas as pd

def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token in operators: operators[token](stack)
        else: stack.push(float(token))
    return stack.peek()

expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

df = pd.DataFrame({'expression': expressions})
df['result'] = df['expression'].apply(lambda x: evaluate_postfix(x))
print(df)

              expression  result
0                  4 6 -   -2.00
1  4 1 2 9 3 / * + 5 - *    8.00
2              1 2 + 3 -    0.00
3              1 2 - 3 +    2.00
4    10 3 5 * 16 4 - / +   11.25
5         5 3 4 2 - ** *   45.00
6        12 2 4 + / 21 *   42.00
7             1 1 + 2 **    4.00
8             1 1 2 ** +    2.00


Next, we look at converting infix to postfix expressions. To do this, we'll implement the [Shunting-yard
algorithm](https://en.wikipedia.org/wiki/Shunting_yard_algorithm). The data structure to implement this algorithm is (again) a stack. The algorithm makes use of the operator precedence levels (defined above) and has specific rules for handing parenthesis, and so on. The result is the `infix_to_postfix` function.

In [7]:
def process_opening_parenthesis(stack):
    stack.push("(")

def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(":
        postfix.append(stack.pop())
    stack.pop() # will be an unnecessary '('

def process_operator(stack, postfix, operator):
    while stack.length > 0 and stack.peek() in operators and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)
               
def process_number(postfix, number):
    postfix.append(number)

def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(": process_opening_parenthesis(stack)
        elif token == ")": process_closing_parenthesis(stack, postfix)
        elif token in operators: process_operator(stack, postfix, token)
        else: process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)

infix_to_postfix("3 + (4 / 2)")

'3 4 2 / +'

And finally, we define the `evaluate` function, which evaluates expressions written in infix.

In [8]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

expressions = [
    "1 + 1",
    "1+1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * (9 / 3) - 5)",
    "10 + 3 * 5 / (16 - 4 * 1)",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 **2** 2 **2** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
    " 64/8 * 8"
]

df = pd.DataFrame({'expression': expressions})
df['result'] = df['expression'].apply(lambda x: evaluate(x))
print(df)

                       expression    result
0                           1 + 1      2.00
1                             1+1      2.00
2           1 * ( 2 - ( 1 + 1 ) )      0.00
3      4 * ( 1 + 2 * (9 / 3) - 5)      8.00
4       10 + 3 * 5 / (16 - 4 * 1)     11.25
5   2 * 2 * 2 * 2 * 2 * 2 * 2 * 2    256.00
6               2 **2** 2 **2** 2  65536.00
7           ( 1 - 2 ) / ( 3 - 5 )      0.50
8                       9 / 8 * 8      9.00
9                  64 / ( 8 * 8 )      1.00
10                       64/8 * 8     64.00
